### The below tensorflow code is compatible with Tensorflow v1.x and we're using tensorflow v2.x which would throw multiple exceptions... Hence, we'll use tf.compat.v1.v1_method_name() after disabling the eager execution whenever needed  
(will be needed in very few places so okay)

In [ ]:
# !pip install tensorflow_datasets

In [ ]:
import tensorflow as tf #Deep learning library
import numpy as np #Matrix algebra library

# the below method throws deprecated warnings so will go with the keras mnist dataset instead...
# from tensorflow.examples.tutorials.mnist import input_data #Import training data

# imported the below dataset earlier but don't yet know how to divide it into training and test splits...
# import tensorflow_datasets
# mnist = tensorflow_datasets.load('mnist', split='train', shuffle_files=True)
# mnist = mnist.shuffle(1024).repeat().batch(32)  # refer to https://www.gcptutorials.com/post/load-mnist-data-with-tensorflow-datsets
# for example in mnist.take(1):
#     image, label = example['image'], example['label']
#     print(image.shape)
#     print(label)

# for i in image:
#     plt.imshow(tf.squeeze(i))
#     plt.show()

import pandas as pd #Database management library
import matplotlib.pyplot as plt #Visualization library 
%matplotlib inline  
from sklearn.svm import LinearSVC #For linear image classification
from sklearn.model_selection import GridSearchCV #For hyperparameter optimization

In [ ]:
(trX1, trY1), (teX1, teY1) = tf.keras.datasets.mnist.load_data()
print(trX1.shape)
print(trY1.shape)
print(teX1.shape)
print(teY1.shape)
trX_mnist=trX1.reshape(-1,trX1.shape[1]*trX1.shape[2]).astype(np.float32)
teX_mnist=trX1.reshape(-1,teX1.shape[1]*teX1.shape[2]).astype(np.float32)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

mnist_df = (lambda n=60000: tfds.as_dataframe(ds.take(n), info))

In [ ]:
info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='~/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [ ]:
uni_labels = mnist_df().label.unique()

In [ ]:
uni_labels

array([4, 1, 0, 7, 8, 2, 6, 3, 9, 5])

In [ ]:
# mnist_df(4)[mnist_df(4)['label'] == 4].to_csv(f"/content/drive/MyDrive/RestrictedBoltzmannMachines/4_df.csv")

In [ ]:
for label in uni_labels:
  mnist_df()[mnist_df()['label'] == label].to_csv(f"/content/drive/MyDrive/RestrictedBoltzmannMachines/label_{label}.csv")
  print(f"label_{label}.csv is saved!")

label_4.csv is saved!
label_1.csv is saved!
label_0.csv is saved!
label_7.csv is saved!
label_8.csv is saved!
label_2.csv is saved!
label_6.csv is saved!
label_3.csv is saved!
label_9.csv is saved!
label_5.csv is saved!


In [ ]:
len(trX2)

1000

In [ ]:
(trX3, try3), (teX3, teY3) = tf.keras.datasets.fashion_mnist.load_data()
print(trX3.shape)
print(try3.shape)
print(teX3.shape)
print(teY3.shape)
trX_fashion_mnist=trX3.reshape(-1,trX3.shape[1]*trX3.shape[2]).astype(np.float32)
teX_fashion_mnist=trX3.reshape(-1,teX3.shape[1]*teX3.shape[2]).astype(np.float32)

4431872/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


# Referred to [JosephGatto's GitHub Repo](https://github.com/JosephGatto/Simplified-Restricted-Boltzmann-Machines/blob/master/Restricted_Boltzmann_Machine.ipynb) for building a RBM from scratch

# Restricted Boltzmann Machine Class

### In the train() method of the class RBM, we see the use of placeholder in tensorflow 2.x version (current version) but we might face errors because tensorflow 2.x doesn't have the placeholders property (only tensorflow version 1.x had it), so we need to migrate the tensorflow 1.x code into tensorflow 2.x


In [ ]:
# disabling the eager execution
tf.compat.v1.disable_eager_execution()

In [ ]:
class RBM(object):
    def __init__(self, input_size, output_size, learning_rate, batch_size):
        self.input_size = input_size #Size of the input layer
        self.output_size = output_size #Size of the hidden layer
        self.epochs = 2 #How many times we will update the weights 
        self.learning_rate = learning_rate #How big of a weight update we will perform 
        self.batch_size = batch_size #How many images will we "feature engineer" at at time 
        self.new_input_layer = None #Initalize new input layer variable for k-step contrastive divergence 
        self.new_hidden_layer = None
        self.new_test_hidden_layer = None
        
        #Here we initialize the weights and biases of our RBM
        #If you are wondering, the 0 ixzs the mean of the distribution we are getting our random weights from. 
        #The .01 is the standard deviation.
        self.w = np.random.normal(0,.01,[input_size,output_size]) #weights
        self.hb = np.random.normal(0,.01,[output_size]) #hidden layer bias
        self.vb = np.random.normal(0,.01,[input_size]) #input layer bias (sometimes called visible layer)
        
        
        #Calculates the sigmoid probabilities of input * weights + bias
        #Here we multiply the input layer by the weights and add the bias
        #This is the phase that creates the hidden layer
    def prob_h_given_v(self, visible, w, hb):
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)
        
        #Calculates the sigmoid probabilities of input * weights + bias
        #Here we multiply the hidden layer by the weights and add the input layer bias
        #This is the reconstruction phase that recreates the original image from the hidden layer
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    #Returns new layer binary values
    #This function returns a 0 or 1 based on the sign of the probabilities passed to it
    #Our RBM will be utilizing binary features to represent the images
    #This function just converts the features we have learned into a binary representation 
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))  # tf.random_uniform -> tf.random.uniform
    
    def train(self, X,extract_weights=None):
        #Initalize placeholder values for graph
        #If this looks strange to you, then you have not used Tensorflow before
        _w = tf.compat.v1.placeholder(tf.float32, shape = [self.input_size, self.output_size])
        _vb = tf.compat.v1.placeholder(tf.float32, shape = [self.input_size])
        _hb = tf.compat.v1.placeholder(tf.float32, shape = [self.output_size])
        
        
        #initalize previous variables
        #we will be saving the weights of the previous and current iterations
        pre_w = np.random.normal(0,.01, size = [self.input_size,self.output_size])
        pre_vb = np.random.normal(0, .01, size = [self.input_size])
        pre_hb = np.random.normal(0, .01, size = [self.output_size])
        
        #initalize current variables
        #we will be saving the weights of the previous and current iterations
        cur_w = np.random.normal(0, .01, size = [self.input_size,self.output_size])
        cur_vb = np.random.normal(0, .01, size = [self.input_size])
        cur_hb = np.random.normal(0, .01, size = [self.output_size])
               
        #Plaecholder variable for input layer
        v0 = tf.compat.v1.placeholder(tf.float32, shape = [None, self.input_size])
        
        #pass probabilities of input * w + b into sample prob to get binary values of hidden layer
        h0 = self.sample_prob(self.prob_h_given_v(v0, _w, _hb ))
        
        #pass probabilities of new hidden unit * w + b into sample prob to get new reconstruction
        v1 = self.sample_prob(self.prob_v_given_h(h0, _w, _vb))
        
        #Just get the probailities of the next hidden layer. We wont need the binary values. 
        #The probabilities here help calculate the gradients during back prop 
        h1 = self.prob_h_given_v(v1, _w, _hb)
        
        
        #Contrastive Divergence
        positive_grad = tf.matmul(tf.transpose(v0), h0) #input' * hidden0
        negative_grad = tf.matmul(tf.transpose(v1), h1) #reconstruction' * hidden1
        #(pos_grad - neg_grad) / total number of input samples 
        CD = (positive_grad - negative_grad) / tf.cast(tf.shape(v0)[0], tf.float32)   # tf.to_float(tf.shape(v0)[0])  -> tf.cast(tf.shape(v0)[0], tf.float32)
        
        #This is just the definition of contrastive divergence 
        update_w = _w + self.learning_rate * CD
        update_vb = _vb + tf.reduce_mean(v0 - v1, 0)
        update_hb = _hb + tf.reduce_mean(h0 - h1, 0)
        
        #MSE - This is our error function
        err = tf.reduce_mean(tf.square(v0 - v1))
        
        #Will hold new visible layer.
        errors = []
        hidden_units = []
        reconstruction = []
        
        test_hidden_units = []
        test_reconstruction=[]
        
        
        #The next four lines of code intitalize our Tensorflow graph and create mini batches
        #The mini batch code is from cognitive class. I love the way they did this. Just giving credit! 
        with tf.compat.v1.Session() as sess:  # now it's compatible with v1
            sess.run(tf.compat.v1.global_variables_initializer())
            for epoch in range(self.epochs):
                for start, end in zip(range(0, len(X), self.batch_size), range(self.batch_size, len(X), self.batch_size)):
                    batch = X[start:end] #Mini batch of images taken from training data
                    
                    #Feed in batch, previous weights/bias, update weights and store them in current weights
                    cur_w = sess.run(update_w, feed_dict = {v0:batch, _w:pre_w , _vb:pre_vb, _hb:pre_hb})
                    cur_hb = sess.run(update_hb, feed_dict = {v0:batch, _w:pre_w , _vb:pre_vb, _hb:pre_hb})
                    cur_vb = sess.run(update_vb, feed_dict = {v0:batch, _w:pre_w , _vb:pre_vb, _hb:pre_hb})
                    
                    #Save weights 
                    pre_w = cur_w
                    pre_hb = cur_hb
                    pre_vb = cur_vb
                
                #At the end of each iteration, the reconstructed images are stored and the error is outputted 
                reconstruction.append(sess.run(v1, feed_dict={v0: X, _w: cur_w, _vb: cur_vb, _hb: cur_hb}))        
                print(f"Learning Rate: {self.learning_rate}:  Batch Size: {self.batch_size}:  Hidden Layers: {self.output_size}: Epoch: {(epoch+1)}: Error: {sess.run(err, feed_dict={v0: X, _w: cur_w, _vb: cur_vb, _hb: cur_hb})}:")
            
            #Store final reconstruction in RBM object
            self.new_input_layer = reconstruction[-1]
            
            #Store weights in RBM object
            self.w = pre_w
            self.hb = pre_hb
            self.vb = pre_vb

            if extract_weights == 1:
                return (self.w, self.hb, self.vb)
    
    #This is used for Contrastive Divergence.
    #This function makes the reconstruction your new input layer. 
    def rbm_output(self, X):
        input_x = tf.constant(X)
        _w = tf.constant(self.w,dtype=tf.float32)
        _hb = tf.constant(self.hb,dtype=tf.float32)
        _vb = tf.constant(self.vb,dtype=tf.float32)

        out = tf.nn.sigmoid(tf.matmul(input_x, _w) + _hb)
        with tf.compat.v1.Session() as sess:
            sess.run(tf.compat.v1.global_variables_initializer())
            return sess.run(out)

In [ ]:
class train_cdk:
  def __init__(self,k,input_size, output_size, learning_rate,
               batch_size,weights=None):
    self.k=k
    self.cdk_train_input=None
    self.cdk_test_input=None
    self.cdk_train_hidden = None #Variable to store hidden layer for training data
    self.cdk_test_hidden = None  #Variable to store hidden layer for testing data
    self.batch_size=batch_size
  
    self.rbm = RBM(input_size, output_size, learning_rate, batch_size)
    self.weights=weights
  def as_w(self):
      self.rbm.w=self.weights
  def ex_w(self,trX):
    self.cdk_train_input = trX
    return self.rbm.train(self.cdk_train_input,1)
  def train(self,trX,teX):   
    self.cdk_train_input = trX #Training data
    self.cdk_test_input = teX  #Testing data 
    #Loop for contrastive divergence
    for i in range(self.k):
        print(f'CD: {int(i+1)}')
        self.rbm.train(self.cdk_train_input) #Using reconstruction as input layer for CD
        self.cdk_train_input = self.rbm.new_input_layer
        self.cdk_train_hidden = self.rbm.rbm_output(self.cdk_train_input)
        self.cdk_test_hidden = self.rbm.rbm_output(self.cdk_test_input)
    return [self.cdk_train_hidden, self.cdk_test_hidden, self.cdk_train_input]
  def get_weights(self):
    return self.rbm.w
  def update_weights(self,weight):
    self.rbm.w=weight

In [ ]:
from sklearn.decomposition import PCA
def compress(weights,r=0):
  nComp=100
  mu = np.mean(weights, axis=0)
  pca=PCA(n_components=nComp).fit(weights)
  compressed=pca.transform(weights)
  if r==1:
    return pca.inverse_transform(compressed)
  return compressed

In [ ]:
trY1.shape[1]

IndexError: ignored

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
#from tensorflow.examples.tutorials.mnist import input_data


#mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 32
z_dim = 10
X_dim = trX1.shape[1]
y_dim = trY1.shape[0]
h_dim = 128
c = 0
lr = 1e-3


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random.normal(shape=size, stddev=xavier_stddev)


""" Q(z|X) """
X = tf.compat.v1.placeholder(tf.float32, shape=[None, X_dim])
z = tf.compat.v1.placeholder(tf.float32, shape=[None, z_dim])

Q_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2 = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2 = tf.Variable(tf.zeros(shape=[z_dim]))

theta_Q = [Q_W1, Q_W2, Q_b1, Q_b2]


def Q(X):
    h = tf.nn.relu(tf.matmul(X, Q_W1) + Q_b1)
    z = tf.matmul(h, Q_W2) + Q_b2
    return z


""" P(X|z) """
P_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_P = [P_W1, P_W2, P_b1, P_b2]


def P(z):
    h = tf.nn.relu(tf.matmul(z, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits


""" D(z) """
D_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


def D(z):
    h = tf.nn.relu(tf.matmul(z, D_W1) + D_b1)
    logits = tf.matmul(h, D_W2) + D_b2
    prob = tf.nn.sigmoid(logits)
    return prob


""" Training """
z_sample = Q(X)
_, logits = P(z_sample)

# Sample from random z
X_samples, _ = P(z)

# E[log P(X|z)]
recon_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X))

# Adversarial loss to approx. Q(z|X)
D_real = D(z)
D_fake = D(z_sample)

D_loss = -tf.reduce_mean(tf.tf.math.log(D_real) + tf.tf.math.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.tf.math.log(D_fake))

AE_solver = tf.train.AdamOptimizer().minimize(recon_loss, var_list=theta_P + theta_Q)
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_Q)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(1000000):
    X_mb, _ = mnist.train.next_batch(mb_size)
    z_mb = np.random.randn(mb_size, z_dim)

    _, recon_loss_curr = sess.run([AE_solver, recon_loss], feed_dict={X: X_mb})
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, z: z_mb})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={X: X_mb})

    if it % 1000 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}; Recon_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr, recon_loss_curr))

        samples = sess.run(X_samples, feed_dict={z: np.random.randn(16, z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

AttributeError: ignored

In [ ]:
#Net 1- initialise
def train_500(input):
  W_500=[]
  with tf.Graph().as_default():
    rbm = train_cdk(1,784,650,.001,32)
  for i in range(500,input.shape[0]+500,500):
    print(f'Iteration{i/500}')
    trX_sampled=input[(i-500):i]
    w=rbm.ex_w(trX_sampled)
    W=np.array(w[0])
    pca_w=compress(W)
    W_500.append(pca_w)
  W=np.array(W_500)
  print(W.shape)
  return W

In [ ]:
w=train_500(trX_mnist)

Iteration1.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 1: Error: 6935.02685546875:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 2: Error: 6935.0166015625:
Iteration2.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 1: Error: 7183.2490234375:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 2: Error: 7183.24853515625:
Iteration3.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 1: Error: 7593.73193359375:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 2: Error: 7593.7255859375:
Iteration4.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 1: Error: 7058.8056640625:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 2: Error: 7058.8017578125:
Iteration5.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 1: Error: 7351.345703125:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 650: Epoch: 2: Error: 7351.33593

In [ ]:
##Net1
def train_net1(n1_w,g=False):
  n1_w=[]
  final_weight=None
  with tf.Graph().as_default():
    rbm = train_cdk(1,784,650,.001,32)
  w=n1_w.reshape(120,-1)
  for i in range(10,w.shape[0]+10,10):
    print(f"Iteration {i/10}")
    temp=rbm.ex_w(w[i-10:i])
    W=np.array(temp[0])
    if (g==True) and (w.shape-i<=10):
      return compress(W,1)
    pca_w=compress(W)
    n1_w.append(pca_w)
  w_2=np.array(n1_w)
  print("End of training of net 1")
  return w_2

In [ ]:
def train_rbm(weights):
  if 10**3>=weights.shape[0]>=10**2:
    n1_w1=train_net1(weights,True)
    return n1_w1
  elif 10**4>=weights.shape[0]>=10**3:
    n1_w1=train_net1(weights)
    n1_w2=train_net1(n1_w1,True)
    return n1_w2
  elif 10**5>weights.shape[0]>=10**4:
    n1_w1=train_net1(weights)
    n1_w2=train_net1(n1_w1)
    n1_w3=train_net1(n1_w2,True)
    return n1_w3
  elif weights.shape[0]>=10**5:
    n1_w1=train_net1(weights)
    n1_w2=train_net1(n1_w1)
    n1_w3=train_net1(n1_w2)
    n1_w4=train_net1(n1_w2,True)
    return n1_w4

In [ ]:
##Net 2
class Net2:
  def __init__(self,input,test,n):
    self.train=input
    self.test=test
    self.batch_size=self.train.shape[0]
    self.n2=n
  def train_net2(self):
    with tf.Graph().as_default():
      rbm_1= train_cdk(1,784,650,.001,self.batch_size,self.n2)
      #assign net 1 weights
      784,1 or 784,650
      rbm_1.as_w()
      #extract weights after training on trX
      temp_1=rbm_1.ex_w(self.train)
      #reassign the previous weights
      rbm_1.as_w()
      W1=np.array(temp_1[0])
      pca_w1,_=compress(W1)
      
      rbm_2= train_cdk(1,784,1,.001,self.batch_size,self.n2)
      rbm_2.as_w()
      #extract weights after training on compressed weights
      temp_2=rbm_2.ex_w(np.transpose(pca_w1))
      #reassign the previous weights
      rbm_2.as_w()
      W2=np.array(temp_2[0])
      pca_w2,t2=compress(W2)
      
      rbm_3= train_cdk(1,784,650,.001,self.batch_size,self.n2)
      rbm_3.as_w()
      #extract weights after training on compressed weights
      temp_3=rbm_3.ex_w(np.transpose(pca_w2))
      #reassign the previous weights
      rbm_3.as_w()
      W3=np.array(temp_3[0])
      pca_w3,t3=compress(W2)

      rbm_4= train_cdk(1,784,650,.001,self.batch_size,self.n2)
      rbm_4.as_w()
      #extract weights after training on compressed weights
      temp_4=rbm_4.ex_w(np.transpose(pca_w3))
      #reassign the previous weights
      rbm_4.as_w()
      W4=np.array(temp_4[0])
      pca_w4,t4=compress(W4)
      
      rbm_5= train_cdk(1,784,650,.001,self.batch_size,self.n2)
      rbm_5.as_w()
      #extract weights after training on compressed weights
      temp_5=rbm_5.ex_w(np.transpose(pca_w4))
      #reassign the previous weights
      rbm_5.as_w()
      W5=np.array(temp_5[0])
      pca_w5,t5=compress(W5)
      #update weights of previous rbm
      rbm_4.update_weights(rbm_5.get_weights())
      #reconstructing using rbm_4
      re_4=rbm_4.train(np.transpose(t4),self.test)
      #update weights of previous rbm
      rbm_3.update_weights(re_4[0])
      #reconstructing using rbm_3
      re_3=rbm_3.train(np.transpose(t3),self.test)
      #update weights of previous rbm
      rbm_2.update_weights(re_3[0])
      #reconstructing using rbm_2
      re_2=rbm_2.train(np.transpose(t2),self.test)
      #update weights of previous rbm
      rbm_1.update_weights(re_2[0])
      rbm_1.rbm.batch_size=32
      re_1=rbm_1.train(self.train,self.test)
      return re_1     

In [ ]:
def train_all(train,test):
  n1_w=train_500(train,test)
  n2=train_rbm(n1_w)
  c=Net2(train,test,n2)
  r=c.train_net2()
  return r

In [ ]:
Final_output=train_all(trX_mnist,teX_mnist)

Iteration1.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 1: Error: 6935.0244140625:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 2: Error: 6935.0166015625:
Iteration2.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 1: Error: 7183.25146484375:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 2: Error: 7183.24365234375:
Iteration3.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 1: Error: 7593.73388671875:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 2: Error: 7593.7255859375:
Iteration4.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 1: Error: 7058.80419921875:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 2: Error: 7058.7998046875:
Iteration5.0
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 1: Error: 7351.34423828125:
Learning Rate: 0.001:  Batch Size: 32:  Hidden Layers: 684: Epoch: 2: Error: 7351.33